# Install requirements

In [ ]:
%%capture
!pip install datasets
!pip install evaluate
!pip install python-dotenv

In [ ]:
import requests
import json
import re
import io
import gc
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict
import huggingface_hub
import os
from dotenv import load_dotenv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer, DefaultDataCollator, pipeline
import evaluate
import numpy as np
from PIL import Image

In [ ]:
load_dotenv()
hf_token = os.getenv(HUGGING_FACE_TOKEN)
decodo_auth = os.getenv(DECODO_AUTH)

# E-Commerce Categories


We are going to be crawling for stock images of the following products:

1. Electronics
2. Fashion
3. Beauty
4. Home
5. Sports
6. Health
7. Toys
8. Automotive
9. Books
10. Jewelry
11. Pet Supplies
12. Baby Products
13. Office Supplies
14. Tools & Hardware
15. Garden & Outdoor
16. Musical Instruments
17. Grocery & Food
18. Arts & Crafts
19. Camera & Photo
20. Smart Home
21. Luggage & Travel
22. Computer Components
23. Cleaning Supplies
24. Industrial & Scientific
25. Party Supplies
26. Wedding
27. School Supplies
28. Seasonal Decorations
29. Swimming Pool
30. Wireless Accessories
31. Maternity
32. Men's Grooming
33. Bathroom Accessories
34. Lighting
35. Bedding & Bath
36. Watches
37. Costumes & Cosplay
38. Sunglasses & Eyewear
39. Collectibles
40. Outdoor Furniture
41. Storage & Organization
42. Candles & Fragrance
43. Sewing & Fabric
44. Puzzles & Brain Teasers
45. Kitchen Gadgets
46. Exercise Equipment
47. Religious & Spiritual
48. Electronic Accessories
49. Maps & Navigation
50. Magazines

In [ ]:
urls_to_crawl = {
    # 1. ELECTRONICS - phones, laptops, headphones
    'electronics': [
        'https://www.amazon.com/s?k=smartphones',
        'https://www.amazon.com/s?k=iphone',
        'https://www.amazon.com/s?k=android+phone',
        'https://www.amazon.com/s?k=samsung+galaxy',
        'https://www.amazon.com/s?k=laptops',
        'https://www.amazon.com/s?k=macbook',
        'https://www.amazon.com/s?k=dell+laptop',
        'https://www.amazon.com/s?k=gaming+laptop',
        'https://www.amazon.com/s?k=chromebook',
        'https://www.amazon.com/s?k=headphones',
        'https://www.amazon.com/s?k=wireless+earbuds',
        'https://www.amazon.com/s?k=airpods',
        'https://www.amazon.com/s?k=bluetooth+headphones',
        'https://www.amazon.com/s?k=gaming+headset',
        'https://www.amazon.com/s?k=tablets',
        'https://www.amazon.com/s?k=ipad',
        'https://www.amazon.com/s?k=smartwatch',
        'https://www.amazon.com/s?k=computer+mouse',
        'https://www.amazon.com/s?k=keyboard',
        'https://www.amazon.com/s?k=webcam'
    ],

    # 2. FASHION & APPAREL - clothing, shoes, accessories
    'fashion': [
        'https://www.amazon.com/s?k=sneakers',
        'https://www.amazon.com/s?k=running+shoes',
        'https://www.amazon.com/s?k=boots',
        'https://www.amazon.com/s?k=sandals',
        'https://www.amazon.com/s?k=dress+shoes',
        'https://www.amazon.com/s?k=t+shirts',
        'https://www.amazon.com/s?k=jeans',
        'https://www.amazon.com/s?k=dresses',
        'https://www.amazon.com/s?k=hoodies',
        'https://www.amazon.com/s?k=jackets',
        'https://www.amazon.com/s?k=pants',
        'https://www.amazon.com/s?k=shorts',
        'https://www.amazon.com/s?k=sweaters',
        'https://www.amazon.com/s?k=handbags',
        'https://www.amazon.com/s?k=backpacks',
        'https://www.amazon.com/s?k=sunglasses',
        'https://www.amazon.com/s?k=hats',
        'https://www.amazon.com/s?k=belts',
        'https://www.amazon.com/s?k=scarves',
        'https://www.amazon.com/s?k=wallets'
    ],

    # 3. BEAUTY & PERSONAL CARE - makeup, skincare, hair products
    'beauty': [
        'https://www.amazon.com/s?k=lipstick',
        'https://www.amazon.com/s?k=foundation',
        'https://www.amazon.com/s?k=mascara',
        'https://www.amazon.com/s?k=eyeshadow+palette',
        'https://www.amazon.com/s?k=concealer',
        'https://www.amazon.com/s?k=moisturizer',
        'https://www.amazon.com/s?k=face+serum',
        'https://www.amazon.com/s?k=cleanser',
        'https://www.amazon.com/s?k=sunscreen',
        'https://www.amazon.com/s?k=face+mask',
        'https://www.amazon.com/s?k=shampoo',
        'https://www.amazon.com/s?k=conditioner',
        'https://www.amazon.com/s?k=hair+oil',
        'https://www.amazon.com/s?k=hair+straightener',
        'https://www.amazon.com/s?k=hair+dryer',
        'https://www.amazon.com/s?k=perfume',
        'https://www.amazon.com/s?k=nail+polish',
        'https://www.amazon.com/s?k=makeup+brushes',
        'https://www.amazon.com/s?k=skin+care+set',
        'https://www.amazon.com/s?k=anti+aging+cream'
    ],

    # 4. HOME & KITCHEN - furniture, appliances, decor
    'home': [
        'https://www.amazon.com/s?k=couch',
        'https://www.amazon.com/s?k=dining+table',
        'https://www.amazon.com/s?k=office+chair',
        'https://www.amazon.com/s?k=bed+frame',
        'https://www.amazon.com/s?k=coffee+table',
        'https://www.amazon.com/s?k=kitchen+appliances',
        'https://www.amazon.com/s?k=blender',
        'https://www.amazon.com/s?k=coffee+maker',
        'https://www.amazon.com/s?k=air+fryer',
        'https://www.amazon.com/s?k=microwave',
        'https://www.amazon.com/s?k=cookware+set',
        'https://www.amazon.com/s?k=kitchen+knives',
        'https://www.amazon.com/s?k=home+decor',
        'https://www.amazon.com/s?k=wall+art',
        'https://www.amazon.com/s?k=throw+pillows',
        'https://www.amazon.com/s?k=lamps',
        'https://www.amazon.com/s?k=curtains',
        'https://www.amazon.com/s?k=rugs',
        'https://www.amazon.com/s?k=storage+organizer',
        'https://www.amazon.com/s?k=bedding+set'
    ],

    # 5. SPORTS & OUTDOORS - fitness gear, camping, sports equipment
    'sports': [
        'https://www.amazon.com/s?k=yoga+mat',
        'https://www.amazon.com/s?k=dumbbells',
        'https://www.amazon.com/s?k=resistance+bands',
        'https://www.amazon.com/s?k=fitness+tracker',
        'https://www.amazon.com/s?k=exercise+bike',
        'https://www.amazon.com/s?k=camping+tent',
        'https://www.amazon.com/s?k=sleeping+bag',
        'https://www.amazon.com/s?k=hiking+backpack',
        'https://www.amazon.com/s?k=camping+chair',
        'https://www.amazon.com/s?k=water+bottle',
        'https://www.amazon.com/s?k=basketball',
        'https://www.amazon.com/s?k=soccer+ball',
        'https://www.amazon.com/s?k=tennis+racket',
        'https://www.amazon.com/s?k=golf+clubs',
        'https://www.amazon.com/s?k=bike+helmet',
        'https://www.amazon.com/s?k=protein+shaker',
        'https://www.amazon.com/s?k=running+gear',
        'https://www.amazon.com/s?k=fishing+rod',
        'https://www.amazon.com/s?k=skateboard',
        'https://www.amazon.com/s?k=outdoor+grill'
    ],

    # 6. HEALTH & WELLNESS - supplements, medical devices
    'health': [
        'https://www.amazon.com/s?k=vitamins',
        'https://www.amazon.com/s?k=protein+powder',
        'https://www.amazon.com/s?k=supplements',
        'https://www.amazon.com/s?k=omega+3',
        'https://www.amazon.com/s?k=multivitamin',
        'https://www.amazon.com/s?k=blood+pressure+monitor',
        'https://www.amazon.com/s?k=thermometer',
        'https://www.amazon.com/s?k=pulse+oximeter',
        'https://www.amazon.com/s?k=heating+pad',
        'https://www.amazon.com/s?k=massage+gun',
        'https://www.amazon.com/s?k=first+aid+kit',
        'https://www.amazon.com/s?k=nebulizer',
        'https://www.amazon.com/s?k=digital+scale',
        'https://www.amazon.com/s?k=pill+organizer',
        'https://www.amazon.com/s?k=compression+socks',
        'https://www.amazon.com/s?k=essential+oils',
        'https://www.amazon.com/s?k=meditation+cushion',
        'https://www.amazon.com/s?k=sleep+aid',
        'https://www.amazon.com/s?k=knee+brace',
        'https://www.amazon.com/s?k=glucose+monitor'
    ],

    # 7. TOYS & GAMES - children's toys, board games
    'toys': [
        'https://www.amazon.com/s?k=lego+sets',
        'https://www.amazon.com/s?k=action+figures',
        'https://www.amazon.com/s?k=dolls',
        'https://www.amazon.com/s?k=board+games',
        'https://www.amazon.com/s?k=puzzle',
        'https://www.amazon.com/s?k=remote+control+car',
        'https://www.amazon.com/s?k=stuffed+animals',
        'https://www.amazon.com/s?k=educational+toys',
        'https://www.amazon.com/s?k=building+blocks',
        'https://www.amazon.com/s?k=art+supplies+kids',
        'https://www.amazon.com/s?k=card+games',
        'https://www.amazon.com/s?k=video+games',
        'https://www.amazon.com/s?k=toy+cars',
        'https://www.amazon.com/s?k=play+kitchen',
        'https://www.amazon.com/s?k=musical+toys',
        'https://www.amazon.com/s?k=outdoor+toys',
        'https://www.amazon.com/s?k=science+kit',
        'https://www.amazon.com/s?k=baby+toys',
        'https://www.amazon.com/s?k=chess+set',
        'https://www.amazon.com/s?k=monopoly'
    ],

    # 8. AUTOMOTIVE - car parts, accessories
    'automotive': [
        'https://www.amazon.com/s?k=car+charger',
        'https://www.amazon.com/s?k=car+phone+mount',
        'https://www.amazon.com/s?k=car+air+freshener',
        'https://www.amazon.com/s?k=car+seat+covers',
        'https://www.amazon.com/s?k=car+floor+mats',
        'https://www.amazon.com/s?k=jumper+cables',
        'https://www.amazon.com/s?k=car+emergency+kit',
        'https://www.amazon.com/s?k=car+vacuum',
        'https://www.amazon.com/s?k=dash+cam',
        'https://www.amazon.com/s?k=car+organizer',
        'https://www.amazon.com/s?k=car+battery',
        'https://www.amazon.com/s?k=motor+oil',
        'https://www.amazon.com/s?k=windshield+wipers',
        'https://www.amazon.com/s?k=car+tire+pressure+gauge',
        'https://www.amazon.com/s?k=car+wash+supplies',
        'https://www.amazon.com/s?k=car+led+lights',
        'https://www.amazon.com/s?k=car+bluetooth+adapter',
        'https://www.amazon.com/s?k=car+sun+shade',
        'https://www.amazon.com/s?k=tire+inflator',
        'https://www.amazon.com/s?k=car+steering+wheel+cover'
    ],

    # 9. BOOKS & MEDIA - books, DVDs, vinyl
    'books': [
        'https://www.amazon.com/s?k=fiction+books',
        'https://www.amazon.com/s?k=mystery+novels',
        'https://www.amazon.com/s?k=romance+books',
        'https://www.amazon.com/s?k=science+fiction+books',
        'https://www.amazon.com/s?k=biography',
        'https://www.amazon.com/s?k=self+help+books',
        'https://www.amazon.com/s?k=cookbooks',
        'https://www.amazon.com/s?k=children+books',
        'https://www.amazon.com/s?k=textbooks',
        'https://www.amazon.com/s?k=business+books',
        'https://www.amazon.com/s?k=dvd+movies',
        'https://www.amazon.com/s?k=blu+ray+movies',
        'https://www.amazon.com/s?k=tv+series+dvd',
        'https://www.amazon.com/s?k=vinyl+records',
        'https://www.amazon.com/s?k=cd+music',
        'https://www.amazon.com/s?k=audiobooks',
        'https://www.amazon.com/s?k=manga',
        'https://www.amazon.com/s?k=comic+books',
        'https://www.amazon.com/s?k=poetry+books',
        'https://www.amazon.com/s?k=travel+guides'
    ],

    # 10. JEWELRY & WATCHES - accessories, luxury items
    'jewelry': [
        'https://www.amazon.com/s?k=necklaces',
        'https://www.amazon.com/s?k=earrings',
        'https://www.amazon.com/s?k=bracelets',
        'https://www.amazon.com/s?k=rings',
        'https://www.amazon.com/s?k=watches',
        'https://www.amazon.com/s?k=smartwatch',
        'https://www.amazon.com/s?k=mens+watches',
        'https://www.amazon.com/s?k=womens+watches',
        'https://www.amazon.com/s?k=gold+jewelry',
        'https://www.amazon.com/s?k=silver+jewelry',
        'https://www.amazon.com/s?k=diamond+rings',
        'https://www.amazon.com/s?k=pearl+necklace',
        'https://www.amazon.com/s?k=engagement+rings',
        'https://www.amazon.com/s?k=wedding+bands',
        'https://www.amazon.com/s?k=vintage+jewelry',
        'https://www.amazon.com/s?k=costume+jewelry',
        'https://www.amazon.com/s?k=jewelry+box',
        'https://www.amazon.com/s?k=cufflinks',
        'https://www.amazon.com/s?k=brooch',
        'https://www.amazon.com/s?k=anklets'
    ],

    # 11. PET SUPPLIES - food, toys, accessories for pets
    'pet_supplies': [
        'https://www.amazon.com/s?k=dog+food',
        'https://www.amazon.com/s?k=cat+food',
        'https://www.amazon.com/s?k=pet+toys',
        'https://www.amazon.com/s?k=dog+leash',
        'https://www.amazon.com/s?k=cat+litter',
        'https://www.amazon.com/s?k=pet+bed',
        'https://www.amazon.com/s?k=aquarium',
        'https://www.amazon.com/s?k=bird+cage',
        'https://www.amazon.com/s?k=dog+collar',
        'https://www.amazon.com/s?k=cat+tree',
        'https://www.amazon.com/s?k=pet+carrier',
        'https://www.amazon.com/s?k=dog+treats',
        'https://www.amazon.com/s?k=pet+grooming',
        'https://www.amazon.com/s?k=fish+tank',
        'https://www.amazon.com/s?k=hamster+wheel',
        'https://www.amazon.com/s?k=pet+bowls',
        'https://www.amazon.com/s?k=dog+harness',
        'https://www.amazon.com/s?k=cat+scratching+post',
        'https://www.amazon.com/s?k=reptile+terrarium',
        'https://www.amazon.com/s?k=pet+vitamins'
    ],

    # 12. BABY PRODUCTS - nursery, feeding, clothing
    'baby_products': [
        'https://www.amazon.com/s?k=baby+stroller',
        'https://www.amazon.com/s?k=car+seat+baby',
        'https://www.amazon.com/s?k=baby+monitor',
        'https://www.amazon.com/s?k=baby+bottles',
        'https://www.amazon.com/s?k=diapers',
        'https://www.amazon.com/s?k=baby+crib',
        'https://www.amazon.com/s?k=baby+clothes',
        'https://www.amazon.com/s?k=baby+swing',
        'https://www.amazon.com/s?k=baby+carrier',
        'https://www.amazon.com/s?k=breast+pump',
        'https://www.amazon.com/s?k=baby+bath+tub',
        'https://www.amazon.com/s?k=baby+blanket',
        'https://www.amazon.com/s?k=pacifiers',
        'https://www.amazon.com/s?k=baby+wipes',
        'https://www.amazon.com/s?k=baby+formula',
        'https://www.amazon.com/s?k=high+chair',
        'https://www.amazon.com/s?k=baby+gate',
        'https://www.amazon.com/s?k=diaper+bag',
        'https://www.amazon.com/s?k=baby+toys+0-6+months',
        'https://www.amazon.com/s?k=nursing+pillow'
    ],

    # 13. OFFICE SUPPLIES - stationery, organization
    'office_supplies': [
        'https://www.amazon.com/s?k=printer+paper',
        'https://www.amazon.com/s?k=pens',
        'https://www.amazon.com/s?k=notebooks',
        'https://www.amazon.com/s?k=desk+organizer',
        'https://www.amazon.com/s?k=stapler',
        'https://www.amazon.com/s?k=file+folders',
        'https://www.amazon.com/s?k=whiteboard',
        'https://www.amazon.com/s?k=calculator',
        'https://www.amazon.com/s?k=paper+clips',
        'https://www.amazon.com/s?k=sticky+notes',
        'https://www.amazon.com/s?k=binder',
        'https://www.amazon.com/s?k=label+maker',
        'https://www.amazon.com/s?k=scissors',
        'https://www.amazon.com/s?k=tape+dispenser',
        'https://www.amazon.com/s?k=pencil+case',
        'https://www.amazon.com/s?k=desk+lamp',
        'https://www.amazon.com/s?k=printer+ink',
        'https://www.amazon.com/s?k=envelopes',
        'https://www.amazon.com/s?k=clipboard',
        'https://www.amazon.com/s?k=rubber+stamps'
    ],

    # 14. TOOLS & HARDWARE - power tools, hand tools
    'tools_hardware': [
        'https://www.amazon.com/s?k=drill',
        'https://www.amazon.com/s?k=hammer',
        'https://www.amazon.com/s?k=screwdriver+set',
        'https://www.amazon.com/s?k=wrench+set',
        'https://www.amazon.com/s?k=saw',
        'https://www.amazon.com/s?k=toolbox',
        'https://www.amazon.com/s?k=tape+measure',
        'https://www.amazon.com/s?k=level+tool',
        'https://www.amazon.com/s?k=pliers',
        'https://www.amazon.com/s?k=socket+set',
        'https://www.amazon.com/s?k=work+gloves',
        'https://www.amazon.com/s?k=ladder',
        'https://www.amazon.com/s?k=multitool',
        'https://www.amazon.com/s?k=sandpaper',
        'https://www.amazon.com/s?k=paint+brushes',
        'https://www.amazon.com/s?k=safety+goggles',
        'https://www.amazon.com/s?k=stud+finder',
        'https://www.amazon.com/s?k=allen+wrench+set',
        'https://www.amazon.com/s?k=utility+knife',
        'https://www.amazon.com/s?k=air+compressor'
    ],

    # 15. GARDEN & OUTDOOR LIVING - plants, tools, furniture
    'garden_outdoor': [
        'https://www.amazon.com/s?k=garden+hose',
        'https://www.amazon.com/s?k=lawn+mower',
        'https://www.amazon.com/s?k=plant+pots',
        'https://www.amazon.com/s?k=garden+tools',
        'https://www.amazon.com/s?k=outdoor+furniture',
        'https://www.amazon.com/s?k=patio+umbrella',
        'https://www.amazon.com/s?k=barbecue+grill',
        'https://www.amazon.com/s?k=fire+pit',
        'https://www.amazon.com/s?k=garden+gloves',
        'https://www.amazon.com/s?k=sprinkler',
        'https://www.amazon.com/s?k=fertilizer',
        'https://www.amazon.com/s?k=outdoor+lights',
        'https://www.amazon.com/s?k=hammock',
        'https://www.amazon.com/s?k=bird+feeder',
        'https://www.amazon.com/s?k=garden+shed',
        'https://www.amazon.com/s?k=leaf+blower',
        'https://www.amazon.com/s?k=pruning+shears',
        'https://www.amazon.com/s?k=wheelbarrow',
        'https://www.amazon.com/s?k=outdoor+cushions',
        'https://www.amazon.com/s?k=gazebo'
    ],

    # 16. MUSICAL INSTRUMENTS - instruments and accessories
    'musical_instruments': [
        'https://www.amazon.com/s?k=guitar',
        'https://www.amazon.com/s?k=piano+keyboard',
        'https://www.amazon.com/s?k=drums',
        'https://www.amazon.com/s?k=violin',
        'https://www.amazon.com/s?k=ukulele',
        'https://www.amazon.com/s?k=microphone',
        'https://www.amazon.com/s?k=guitar+picks',
        'https://www.amazon.com/s?k=music+stand',
        'https://www.amazon.com/s?k=saxophone',
        'https://www.amazon.com/s?k=trumpet',
        'https://www.amazon.com/s?k=flute',
        'https://www.amazon.com/s?k=harmonica',
        'https://www.amazon.com/s?k=bass+guitar',
        'https://www.amazon.com/s?k=guitar+amp',
        'https://www.amazon.com/s?k=capo',
        'https://www.amazon.com/s?k=metronome',
        'https://www.amazon.com/s?k=guitar+strings',
        'https://www.amazon.com/s?k=drum+sticks',
        'https://www.amazon.com/s?k=piano+bench',
        'https://www.amazon.com/s?k=music+sheet'
    ],

    # 17. GROCERY & GOURMET FOOD - snacks, beverages, specialty foods
    'grocery_food': [
        'https://www.amazon.com/s?k=coffee+beans',
        'https://www.amazon.com/s?k=tea',
        'https://www.amazon.com/s?k=chocolate',
        'https://www.amazon.com/s?k=snacks',
        'https://www.amazon.com/s?k=cookies',
        'https://www.amazon.com/s?k=candy',
        'https://www.amazon.com/s?k=protein+bars',
        'https://www.amazon.com/s?k=nuts',
        'https://www.amazon.com/s?k=dried+fruit',
        'https://www.amazon.com/s?k=pasta',
        'https://www.amazon.com/s?k=olive+oil',
        'https://www.amazon.com/s?k=honey',
        'https://www.amazon.com/s?k=spices',
        'https://www.amazon.com/s?k=hot+sauce',
        'https://www.amazon.com/s?k=energy+drinks',
        'https://www.amazon.com/s?k=granola',
        'https://www.amazon.com/s?k=popcorn',
        'https://www.amazon.com/s?k=beef+jerky',
        'https://www.amazon.com/s?k=cereal',
        'https://www.amazon.com/s?k=jam'
    ],

    # 18. ARTS & CRAFTS - art supplies, craft materials
    'arts_crafts': [
        'https://www.amazon.com/s?k=paint+set',
        'https://www.amazon.com/s?k=canvas',
        'https://www.amazon.com/s?k=colored+pencils',
        'https://www.amazon.com/s?k=markers',
        'https://www.amazon.com/s?k=watercolors',
        'https://www.amazon.com/s?k=sketch+pad',
        'https://www.amazon.com/s?k=easel',
        'https://www.amazon.com/s?k=yarn',
        'https://www.amazon.com/s?k=knitting+needles',
        'https://www.amazon.com/s?k=sewing+kit',
        'https://www.amazon.com/s?k=fabric',
        'https://www.amazon.com/s?k=glue+gun',
        'https://www.amazon.com/s?k=beads',
        'https://www.amazon.com/s?k=scrapbook',
        'https://www.amazon.com/s?k=clay',
        'https://www.amazon.com/s?k=calligraphy+pen',
        'https://www.amazon.com/s?k=origami+paper',
        'https://www.amazon.com/s?k=craft+scissors',
        'https://www.amazon.com/s?k=stencils',
        'https://www.amazon.com/s?k=embroidery+kit'
    ],

    # 19. CAMERA & PHOTO - cameras, lenses, accessories
    'camera_photo': [
        'https://www.amazon.com/s?k=dslr+camera',
        'https://www.amazon.com/s?k=mirrorless+camera',
        'https://www.amazon.com/s?k=camera+lens',
        'https://www.amazon.com/s?k=tripod',
        'https://www.amazon.com/s?k=camera+bag',
        'https://www.amazon.com/s?k=memory+card',
        'https://www.amazon.com/s?k=camera+flash',
        'https://www.amazon.com/s?k=action+camera',
        'https://www.amazon.com/s?k=instant+camera',
        'https://www.amazon.com/s?k=camera+battery',
        'https://www.amazon.com/s?k=lens+filter',
        'https://www.amazon.com/s?k=camera+strap',
        'https://www.amazon.com/s?k=light+reflector',
        'https://www.amazon.com/s?k=backdrop+stand',
        'https://www.amazon.com/s?k=ring+light',
        'https://www.amazon.com/s?k=camera+cleaning+kit',
        'https://www.amazon.com/s?k=gimbal+stabilizer',
        'https://www.amazon.com/s?k=photo+printer',
        'https://www.amazon.com/s?k=drone+camera',
        'https://www.amazon.com/s?k=binoculars'
    ],

    # 20. SMART HOME - automation, security, assistants
    'smart_home': [
        'https://www.amazon.com/s?k=smart+speaker',
        'https://www.amazon.com/s?k=smart+light+bulb',
        'https://www.amazon.com/s?k=smart+thermostat',
        'https://www.amazon.com/s?k=smart+door+lock',
        'https://www.amazon.com/s?k=security+camera',
        'https://www.amazon.com/s?k=video+doorbell',
        'https://www.amazon.com/s?k=smart+plug',
        'https://www.amazon.com/s?k=smart+switch',
        'https://www.amazon.com/s?k=motion+sensor',
        'https://www.amazon.com/s?k=smart+hub',
        'https://www.amazon.com/s?k=smart+display',
        'https://www.amazon.com/s?k=wifi+router',
        'https://www.amazon.com/s?k=smart+smoke+detector',
        'https://www.amazon.com/s?k=smart+garage+door',
        'https://www.amazon.com/s?k=smart+curtains',
        'https://www.amazon.com/s?k=water+leak+sensor',
        'https://www.amazon.com/s?k=smart+outlet',
        'https://www.amazon.com/s?k=home+automation',
        'https://www.amazon.com/s?k=smart+doorbell'
    ],

    # 21. LUGGAGE & TRAVEL - suitcases, travel accessories
    'luggage_travel': [
        'https://www.amazon.com/s?k=suitcase',
        'https://www.amazon.com/s?k=carry+on+luggage',
        'https://www.amazon.com/s?k=travel+backpack',
        'https://www.amazon.com/s?k=duffel+bag',
        'https://www.amazon.com/s?k=luggage+set',
        'https://www.amazon.com/s?k=travel+pillow',
        'https://www.amazon.com/s?k=luggage+tags',
        'https://www.amazon.com/s?k=packing+cubes',
        'https://www.amazon.com/s?k=travel+adapter',
        'https://www.amazon.com/s?k=passport+holder',
        'https://www.amazon.com/s?k=toiletry+bag',
        'https://www.amazon.com/s?k=luggage+scale',
        'https://www.amazon.com/s?k=garment+bag',
        'https://www.amazon.com/s?k=travel+bottles',
        'https://www.amazon.com/s?k=neck+wallet',
        'https://www.amazon.com/s?k=travel+blanket',
        'https://www.amazon.com/s?k=compression+bags',
        'https://www.amazon.com/s?k=luggage+lock',
        'https://www.amazon.com/s?k=travel+umbrella',
        'https://www.amazon.com/s?k=travel+organizer'
    ],

    # 22. COMPUTER COMPONENTS - PC parts, gaming hardware
    'computer_components': [
        'https://www.amazon.com/s?k=graphics+card',
        'https://www.amazon.com/s?k=motherboard',
        'https://www.amazon.com/s?k=cpu+processor',
        'https://www.amazon.com/s?k=ram+memory',
        'https://www.amazon.com/s?k=ssd+drive',
        'https://www.amazon.com/s?k=power+supply+unit',
        'https://www.amazon.com/s?k=computer+case',
        'https://www.amazon.com/s?k=cpu+cooler',
        'https://www.amazon.com/s?k=case+fans',
        'https://www.amazon.com/s?k=thermal+paste',
        'https://www.amazon.com/s?k=monitor',
        'https://www.amazon.com/s?k=gaming+monitor',
        'https://www.amazon.com/s?k=mechanical+keyboard',
        'https://www.amazon.com/s?k=gaming+mouse',
        'https://www.amazon.com/s?k=mouse+pad',
        'https://www.amazon.com/s?k=usb+hub',
        'https://www.amazon.com/s?k=hdmi+cable',
        'https://www.amazon.com/s?k=display+port+cable',
        'https://www.amazon.com/s?k=sound+card',
        'https://www.amazon.com/s?k=network+card'
    ],

    # 23. CLEANING SUPPLIES - household cleaning products
    'cleaning_supplies': [
        'https://www.amazon.com/s?k=vacuum+cleaner',
        'https://www.amazon.com/s?k=robot+vacuum',
        'https://www.amazon.com/s?k=mop',
        'https://www.amazon.com/s?k=broom',
        'https://www.amazon.com/s?k=cleaning+spray',
        'https://www.amazon.com/s?k=disinfectant+wipes',
        'https://www.amazon.com/s?k=laundry+detergent',
        'https://www.amazon.com/s?k=dish+soap',
        'https://www.amazon.com/s?k=sponges',
        'https://www.amazon.com/s?k=microfiber+cloths',
        'https://www.amazon.com/s?k=trash+bags',
        'https://www.amazon.com/s?k=cleaning+gloves',
        'https://www.amazon.com/s?k=toilet+cleaner',
        'https://www.amazon.com/s?k=glass+cleaner',
        'https://www.amazon.com/s?k=carpet+cleaner',
        'https://www.amazon.com/s?k=steam+cleaner',
        'https://www.amazon.com/s?k=duster',
        'https://www.amazon.com/s?k=scrub+brush',
        'https://www.amazon.com/s?k=air+freshener',
        'https://www.amazon.com/s?k=fabric+softener'
    ],

    # 24. INDUSTRIAL & SCIENTIFIC - lab equipment, safety gear
    'industrial_scientific': [
        'https://www.amazon.com/s?k=microscope',
        'https://www.amazon.com/s?k=telescope',
        'https://www.amazon.com/s?k=lab+gloves',
        'https://www.amazon.com/s?k=safety+helmet',
        'https://www.amazon.com/s?k=lab+coat',
        'https://www.amazon.com/s?k=beaker+set',
        'https://www.amazon.com/s?k=test+tubes',
        'https://www.amazon.com/s?k=petri+dishes',
        'https://www.amazon.com/s?k=digital+calipers',
        'https://www.amazon.com/s?k=multimeter',
        'https://www.amazon.com/s?k=soldering+iron',
        'https://www.amazon.com/s?k=heat+gun',
        'https://www.amazon.com/s?k=respirator+mask',
        'https://www.amazon.com/s?k=ear+protection',
        'https://www.amazon.com/s?k=safety+vest',
        'https://www.amazon.com/s?k=work+boots',
        'https://www.amazon.com/s?k=measuring+scale',
        'https://www.amazon.com/s?k=magnifying+glass',
        'https://www.amazon.com/s?k=lab+thermometer',
        'https://www.amazon.com/s?k=ph+meter'
    ],

    # 25. PARTY SUPPLIES - decorations, tableware
    'party_supplies': [
        'https://www.amazon.com/s?k=party+decorations',
        'https://www.amazon.com/s?k=balloons',
        'https://www.amazon.com/s?k=party+plates',
        'https://www.amazon.com/s?k=party+cups',
        'https://www.amazon.com/s?k=birthday+candles',
        'https://www.amazon.com/s?k=party+streamers',
        'https://www.amazon.com/s?k=party+favors',
        'https://www.amazon.com/s?k=piñata',
        'https://www.amazon.com/s?k=confetti',
        'https://www.amazon.com/s?k=party+hats',
        'https://www.amazon.com/s?k=tablecloth',
        'https://www.amazon.com/s?k=party+napkins',
        'https://www.amazon.com/s?k=cake+toppers',
        'https://www.amazon.com/s?k=party+games',
        'https://www.amazon.com/s?k=photo+booth+props',
        'https://www.amazon.com/s?k=party+invitations',
        'https://www.amazon.com/s?k=gift+bags',
        'https://www.amazon.com/s?k=party+banner',
        'https://www.amazon.com/s?k=disposable+utensils',
        'https://www.amazon.com/s?k=party+lights'
    ],

    # 26. WEDDING - wedding accessories and supplies
    'wedding': [
        'https://www.amazon.com/s?k=wedding+decorations',
        'https://www.amazon.com/s?k=wedding+veil',
        'https://www.amazon.com/s?k=wedding+shoes',
        'https://www.amazon.com/s?k=bridesmaid+gifts',
        'https://www.amazon.com/s?k=wedding+guest+book',
        'https://www.amazon.com/s?k=wedding+cake+topper',
        'https://www.amazon.com/s?k=wedding+favors',
        'https://www.amazon.com/s?k=ring+bearer+pillow',
        'https://www.amazon.com/s?k=flower+girl+basket',
        'https://www.amazon.com/s?k=wedding+arch',
        'https://www.amazon.com/s?k=wedding+candles',
        'https://www.amazon.com/s?k=wedding+invitations',
        'https://www.amazon.com/s?k=wedding+card+box',
        'https://www.amazon.com/s?k=bridal+accessories',
        'https://www.amazon.com/s?k=wedding+bouquet',
        'https://www.amazon.com/s?k=wedding+centerpieces',
        'https://www.amazon.com/s?k=unity+candle',
        'https://www.amazon.com/s?k=wedding+sash',
        'https://www.amazon.com/s?k=groomsmen+gifts',
        'https://www.amazon.com/s?k=wedding+sparklers'
    ],

    # 27. SCHOOL SUPPLIES - student essentials
    'school_supplies': [
        'https://www.amazon.com/s?k=school+backpack',
        'https://www.amazon.com/s?k=lunch+box',
        'https://www.amazon.com/s?k=pencil+box',
        'https://www.amazon.com/s?k=crayons',
        'https://www.amazon.com/s?k=school+glue',
        'https://www.amazon.com/s?k=composition+notebook',
        'https://www.amazon.com/s?k=index+cards',
        'https://www.amazon.com/s?k=highlighters',
        'https://www.amazon.com/s?k=erasers',
        'https://www.amazon.com/s?k=ruler',
        'https://www.amazon.com/s?k=protractor',
        'https://www.amazon.com/s?k=compass+geometry',
        'https://www.amazon.com/s?k=book+covers',
        'https://www.amazon.com/s?k=locker+organizer',
        'https://www.amazon.com/s?k=pencil+sharpener',
        'https://www.amazon.com/s?k=school+planner',
        'https://www.amazon.com/s?k=flash+cards',
        'https://www.amazon.com/s?k=pocket+folders',
        'https://www.amazon.com/s?k=graph+paper',
        'https://www.amazon.com/s?k=school+supplies+kit'
    ],

    # 28. SEASONAL DECORATIONS - holiday and seasonal items
    'seasonal_decorations': [
        'https://www.amazon.com/s?k=christmas+tree',
        'https://www.amazon.com/s?k=christmas+lights',
        'https://www.amazon.com/s?k=halloween+decorations',
        'https://www.amazon.com/s?k=halloween+costume',
        'https://www.amazon.com/s?k=easter+decorations',
        'https://www.amazon.com/s?k=thanksgiving+decorations',
        'https://www.amazon.com/s?k=valentine+decorations',
        'https://www.amazon.com/s?k=christmas+ornaments',
        'https://www.amazon.com/s?k=halloween+candy+bowl',
        'https://www.amazon.com/s?k=christmas+stockings',
        'https://www.amazon.com/s?k=wreath',
        'https://www.amazon.com/s?k=string+lights',
        'https://www.amazon.com/s?k=pumpkin+decorations',
        'https://www.amazon.com/s?k=santa+hat',
        'https://www.amazon.com/s?k=easter+basket',
        'https://www.amazon.com/s?k=christmas+garland',
        'https://www.amazon.com/s?k=menorah',
        'https://www.amazon.com/s?k=fireworks',
        'https://www.amazon.com/s?k=new+year+decorations',
        'https://www.amazon.com/s?k=halloween+mask'
    ],

    # 29. SWIMMING POOL & SPA - pool accessories, maintenance
    'swimming_pool': [
        'https://www.amazon.com/s?k=pool+float',
        'https://www.amazon.com/s?k=swimming+goggles',
        'https://www.amazon.com/s?k=pool+toys',
        'https://www.amazon.com/s?k=pool+chemicals',
        'https://www.amazon.com/s?k=pool+cover',
        'https://www.amazon.com/s?k=pool+pump',
        'https://www.amazon.com/s?k=pool+filter',
        'https://www.amazon.com/s?k=pool+vacuum',
        'https://www.amazon.com/s?k=pool+ladder',
        'https://www.amazon.com/s?k=pool+thermometer',
        'https://www.amazon.com/s?k=pool+lights',
        'https://www.amazon.com/s?k=pool+noodles',
        'https://www.amazon.com/s?k=diving+toys',
        'https://www.amazon.com/s?k=pool+basketball',
        'https://www.amazon.com/s?k=pool+volleyball',
        'https://www.amazon.com/s?k=hot+tub+chemicals',
        'https://www.amazon.com/s?k=pool+skimmer',
        'https://www.amazon.com/s?k=pool+test+kit',
        'https://www.amazon.com/s?k=pool+alarm',
        'https://www.amazon.com/s?k=pool+heater'
    ],

    # 30. WIRELESS & ACCESSORIES - phone cases, chargers
    'wireless_accessories': [
        'https://www.amazon.com/s?k=phone+case',
        'https://www.amazon.com/s?k=screen+protector',
        'https://www.amazon.com/s?k=phone+charger',
        'https://www.amazon.com/s?k=wireless+charger',
        'https://www.amazon.com/s?k=power+bank',
        'https://www.amazon.com/s?k=phone+stand',
        'https://www.amazon.com/s?k=car+phone+holder',
        'https://www.amazon.com/s?k=selfie+stick',
        'https://www.amazon.com/s?k=phone+grip',
        'https://www.amazon.com/s?k=usb+cable',
        'https://www.amazon.com/s?k=lightning+cable',
        'https://www.amazon.com/s?k=usb+c+cable',
        'https://www.amazon.com/s?k=phone+lens',
        'https://www.amazon.com/s?k=bluetooth+adapter',
        'https://www.amazon.com/s?k=sim+card',
        'https://www.amazon.com/s?k=phone+wallet',
        'https://www.amazon.com/s?k=phone+armband',
        'https://www.amazon.com/s?k=phone+lanyard',
        'https://www.amazon.com/s?k=phone+cleaning+kit',
        'https://www.amazon.com/s?k=phone+ring+light'
    ],

    # 31. MATERNITY - pregnancy and maternity products
    'maternity': [
        'https://www.amazon.com/s?k=maternity+clothes',
        'https://www.amazon.com/s?k=maternity+dress',
        'https://www.amazon.com/s?k=maternity+jeans',
        'https://www.amazon.com/s?k=maternity+pillow',
        'https://www.amazon.com/s?k=pregnancy+test',
        'https://www.amazon.com/s?k=prenatal+vitamins',
        'https://www.amazon.com/s?k=maternity+belt',
        'https://www.amazon.com/s?k=nursing+bra',
        'https://www.amazon.com/s?k=maternity+leggings',
        'https://www.amazon.com/s?k=belly+band',
        'https://www.amazon.com/s?k=pregnancy+journal',
        'https://www.amazon.com/s?k=stretch+mark+cream',
        'https://www.amazon.com/s?k=maternity+swimsuit',
        'https://www.amazon.com/s?k=nursing+pads',
        'https://www.amazon.com/s?k=maternity+underwear',
        'https://www.amazon.com/s?k=pregnancy+books',
        'https://www.amazon.com/s?k=baby+bump+stickers',
        'https://www.amazon.com/s?k=maternity+coat',
        'https://www.amazon.com/s?k=nursing+cover',
        'https://www.amazon.com/s?k=maternity+support'
    ],

    # 32. MENS GROOMING - shaving, beard care
    'mens_grooming': [
        'https://www.amazon.com/s?k=electric+shaver',
        'https://www.amazon.com/s?k=beard+trimmer',
        'https://www.amazon.com/s?k=razor+blades',
        'https://www.amazon.com/s?k=shaving+cream',
        'https://www.amazon.com/s?k=aftershave',
        'https://www.amazon.com/s?k=beard+oil',
        'https://www.amazon.com/s?k=beard+balm',
        'https://www.amazon.com/s?k=mens+cologne',
        'https://www.amazon.com/s?k=hair+clippers',
        'https://www.amazon.com/s?k=nose+hair+trimmer',
        'https://www.amazon.com/s?k=mens+deodorant',
        'https://www.amazon.com/s?k=body+wash+men',
        'https://www.amazon.com/s?k=face+wash+men',
        'https://www.amazon.com/s?k=mens+hair+gel',
        'https://www.amazon.com/s?k=pomade',
        'https://www.amazon.com/s?k=shaving+brush',
        'https://www.amazon.com/s?k=mens+grooming+kit',
        'https://www.amazon.com/s?k=beard+comb',
        'https://www.amazon.com/s?k=mens+face+moisturizer',
        'https://www.amazon.com/s?k=hair+wax'
    ],

    # 33. BATHROOM ACCESSORIES - shower, bath items
    'bathroom_accessories': [
        'https://www.amazon.com/s?k=shower+curtain',
        'https://www.amazon.com/s?k=bath+mat',
        'https://www.amazon.com/s?k=towels',
        'https://www.amazon.com/s?k=toilet+brush',
        'https://www.amazon.com/s?k=soap+dispenser',
        'https://www.amazon.com/s?k=toothbrush+holder',
        'https://www.amazon.com/s?k=shower+caddy',
        'https://www.amazon.com/s?k=bathroom+scale',
        'https://www.amazon.com/s?k=toilet+paper+holder',
        'https://www.amazon.com/s?k=bathroom+mirror',
        'https://www.amazon.com/s?k=shower+head',
        'https://www.amazon.com/s?k=bath+pillow',
        'https://www.amazon.com/s?k=bathroom+organizer',
        'https://www.amazon.com/s?k=toilet+seat',
        'https://www.amazon.com/s?k=bathroom+trash+can',
        'https://www.amazon.com/s?k=bath+bombs',
        'https://www.amazon.com/s?k=loofa',
        'https://www.amazon.com/s?k=bathroom+rug',
        'https://www.amazon.com/s?k=towel+rack',
        'https://www.amazon.com/s?k=plunger'
    ],

    # 34. LIGHTING - indoor and outdoor lighting
    'lighting': [
        'https://www.amazon.com/s?k=led+bulbs',
        'https://www.amazon.com/s?k=floor+lamp',
        'https://www.amazon.com/s?k=table+lamp',
        'https://www.amazon.com/s?k=ceiling+light',
        'https://www.amazon.com/s?k=chandelier',
        'https://www.amazon.com/s?k=pendant+light',
        'https://www.amazon.com/s?k=wall+sconce',
        'https://www.amazon.com/s?k=desk+lamp',
        'https://www.amazon.com/s?k=night+light',
        'https://www.amazon.com/s?k=fairy+lights',
        'https://www.amazon.com/s?k=led+strip+lights',
        'https://www.amazon.com/s?k=outdoor+solar+lights',
        'https://www.amazon.com/s?k=flood+lights',
        'https://www.amazon.com/s?k=track+lighting',
        'https://www.amazon.com/s?k=lamp+shade',
        'https://www.amazon.com/s?k=light+switch',
        'https://www.amazon.com/s?k=dimmer+switch',
        'https://www.amazon.com/s?k=motion+sensor+light',
        'https://www.amazon.com/s?k=grow+lights',
        'https://www.amazon.com/s?k=bathroom+vanity+light'
    ],

    # 35. BEDDING & BATH - sheets, towels, blankets
    'bedding_bath': [
        'https://www.amazon.com/s?k=bed+sheets',
        'https://www.amazon.com/s?k=comforter',
        'https://www.amazon.com/s?k=duvet+cover',
        'https://www.amazon.com/s?k=pillows',
        'https://www.amazon.com/s?k=pillow+cases',
        'https://www.amazon.com/s?k=mattress+pad',
        'https://www.amazon.com/s?k=mattress+topper',
        'https://www.amazon.com/s?k=weighted+blanket',
        'https://www.amazon.com/s?k=throw+blanket',
        'https://www.amazon.com/s?k=bed+skirt',
        'https://www.amazon.com/s?k=bath+towels',
        'https://www.amazon.com/s?k=hand+towels',
        'https://www.amazon.com/s?k=wash+cloths',
        'https://www.amazon.com/s?k=beach+towels',
        'https://www.amazon.com/s?k=bathrobe',
        'https://www.amazon.com/s?k=quilt',
        'https://www.amazon.com/s?k=electric+blanket',
        'https://www.amazon.com/s?k=silk+pillowcase',
        'https://www.amazon.com/s?k=memory+foam+pillow',
        'https://www.amazon.com/s?k=shower+liner'
    ],

    # 36. WATCHES - all types of watches
    'watches': [
        'https://www.amazon.com/s?k=luxury+watches',
        'https://www.amazon.com/s?k=sports+watch',
        'https://www.amazon.com/s?k=digital+watch',
        'https://www.amazon.com/s?k=analog+watch',
        'https://www.amazon.com/s?k=pocket+watch',
        'https://www.amazon.com/s?k=watch+bands',
        'https://www.amazon.com/s?k=watch+box',
        'https://www.amazon.com/s?k=watch+winder',
        'https://www.amazon.com/s?k=diving+watch',
        'https://www.amazon.com/s?k=military+watch',
        'https://www.amazon.com/s?k=nurse+watch',
        'https://www.amazon.com/s?k=kids+watch',
        'https://www.amazon.com/s?k=wooden+watch',
        'https://www.amazon.com/s?k=chronograph+watch',
        'https://www.amazon.com/s?k=skeleton+watch',
        'https://www.amazon.com/s?k=watch+tools',
        'https://www.amazon.com/s?k=watch+repair+kit',
        'https://www.amazon.com/s?k=watch+battery',
        'https://www.amazon.com/s?k=watch+display+stand',
        'https://www.amazon.com/s?k=vintage+watch'
    ],

    # 37. COSTUMES & COSPLAY - costumes for all occasions
    'costumes_cosplay': [
        'https://www.amazon.com/s?k=cosplay+costume',
        'https://www.amazon.com/s?k=anime+costume',
        'https://www.amazon.com/s?k=superhero+costume',
        'https://www.amazon.com/s?k=princess+costume',
        'https://www.amazon.com/s?k=pirate+costume',
        'https://www.amazon.com/s?k=costume+wigs',
        'https://www.amazon.com/s?k=costume+makeup',
        'https://www.amazon.com/s?k=costume+props',
        'https://www.amazon.com/s?k=costume+accessories',
        'https://www.amazon.com/s?k=medieval+costume',
        'https://www.amazon.com/s?k=zombie+costume',
        'https://www.amazon.com/s?k=vampire+costume',
        'https://www.amazon.com/s?k=witch+costume',
        'https://www.amazon.com/s?k=animal+costume',
        'https://www.amazon.com/s?k=mascot+costume',
        'https://www.amazon.com/s?k=costume+sword',
        'https://www.amazon.com/s?k=costume+armor',
        'https://www.amazon.com/s?k=costume+cape',
        'https://www.amazon.com/s?k=costume+gloves',
        'https://www.amazon.com/s?k=costume+boots'
    ],

    # 38. SUNGLASSES & EYEWEAR - prescription and fashion eyewear
    'sunglasses_eyewear': [
        'https://www.amazon.com/s?k=aviator+sunglasses',
        'https://www.amazon.com/s?k=polarized+sunglasses',
        'https://www.amazon.com/s?k=reading+glasses',
        'https://www.amazon.com/s?k=blue+light+glasses',
        'https://www.amazon.com/s?k=prescription+glasses',
        'https://www.amazon.com/s?k=safety+glasses',
        'https://www.amazon.com/s?k=sports+sunglasses',
        'https://www.amazon.com/s?k=cat+eye+sunglasses',
        'https://www.amazon.com/s?k=round+sunglasses',
        'https://www.amazon.com/s?k=wayfarer+sunglasses',
        'https://www.amazon.com/s?k=glasses+case',
        'https://www.amazon.com/s?k=glasses+cleaner',
        'https://www.amazon.com/s?k=eyeglass+chain',
        'https://www.amazon.com/s?k=clip+on+sunglasses',
        'https://www.amazon.com/s?k=kids+sunglasses',
        'https://www.amazon.com/s?k=vintage+sunglasses',
        'https://www.amazon.com/s?k=oversized+sunglasses',
        'https://www.amazon.com/s?k=mirrored+sunglasses',
        'https://www.amazon.com/s?k=glasses+repair+kit',
        'https://www.amazon.com/s?k=contact+lenses'
    ],

    # 39. COLLECTIBLES - trading cards, memorabilia
    'collectibles': [
        'https://www.amazon.com/s?k=pokemon+cards',
        'https://www.amazon.com/s?k=baseball+cards',
        'https://www.amazon.com/s?k=funko+pop',
        'https://www.amazon.com/s?k=coins+collection',
        'https://www.amazon.com/s?k=stamps+collection',
        'https://www.amazon.com/s?k=vintage+toys',
        'https://www.amazon.com/s?k=sports+memorabilia',
        'https://www.amazon.com/s?k=autographed+items',
        'https://www.amazon.com/s?k=collectible+figurines',
        'https://www.amazon.com/s?k=trading+cards',
        'https://www.amazon.com/s?k=vintage+posters',
        'https://www.amazon.com/s?k=model+cars',
        'https://www.amazon.com/s?k=collectible+knives',
        'https://www.amazon.com/s?k=vintage+comics',
        'https://www.amazon.com/s?k=collectible+dolls',
        'https://www.amazon.com/s?k=pin+collection',
        'https://www.amazon.com/s?k=display+case',
        'https://www.amazon.com/s?k=card+sleeves',
        'https://www.amazon.com/s?k=collectible+coins'
    ],

    # 40. OUTDOOR FURNITURE - patio and garden furniture
    'outdoor_furniture': [
        'https://www.amazon.com/s?k=patio+furniture+set',
        'https://www.amazon.com/s?k=outdoor+chairs',
        'https://www.amazon.com/s?k=patio+table',
        'https://www.amazon.com/s?k=outdoor+bench',
        'https://www.amazon.com/s?k=adirondack+chair',
        'https://www.amazon.com/s?k=outdoor+swing',
        'https://www.amazon.com/s?k=patio+heater',
        'https://www.amazon.com/s?k=outdoor+ottoman',
        'https://www.amazon.com/s?k=outdoor+loveseat',
        'https://www.amazon.com/s?k=picnic+table',
        'https://www.amazon.com/s?k=outdoor+bar+set',
        'https://www.amazon.com/s?k=garden+bench',
        'https://www.amazon.com/s?k=outdoor+rocking+chair',
        'https://www.amazon.com/s?k=patio+cushions',
        'https://www.amazon.com/s?k=outdoor+rug',
        'https://www.amazon.com/s?k=furniture+cover',
        'https://www.amazon.com/s?k=outdoor+storage+box',
        'https://www.amazon.com/s?k=porch+swing',
        'https://www.amazon.com/s?k=outdoor+daybed',
        'https://www.amazon.com/s?k=outdoor+side+table'
    ],

    # 41. STORAGE & ORGANIZATION - boxes, shelving, organizers
    'storage_organization': [
        'https://www.amazon.com/s?k=storage+bins',
        'https://www.amazon.com/s?k=storage+shelves',
        'https://www.amazon.com/s?k=closet+organizer',
        'https://www.amazon.com/s?k=drawer+organizer',
        'https://www.amazon.com/s?k=storage+ottoman',
        'https://www.amazon.com/s?k=plastic+containers',
        'https://www.amazon.com/s?k=storage+baskets',
        'https://www.amazon.com/s?k=under+bed+storage',
        'https://www.amazon.com/s?k=shoe+organizer',
        'https://www.amazon.com/s?k=garage+storage',
        'https://www.amazon.com/s?k=cube+storage',
        'https://www.amazon.com/s?k=hanging+organizer',
        'https://www.amazon.com/s?k=storage+cabinet',
        'https://www.amazon.com/s?k=vacuum+storage+bags',
        'https://www.amazon.com/s?k=jewelry+organizer',
        'https://www.amazon.com/s?k=makeup+organizer',
        'https://www.amazon.com/s?k=pantry+organizer',
        'https://www.amazon.com/s?k=file+cabinet',
        'https://www.amazon.com/s?k=storage+trunk',
        'https://www.amazon.com/s?k=wall+organizer'
    ],

    # 42. CANDLES & HOME FRAGRANCE - candles, diffusers, incense
    'candles_fragrance': [
        'https://www.amazon.com/s?k=scented+candles',
        'https://www.amazon.com/s?k=candle+holders',
        'https://www.amazon.com/s?k=essential+oil+diffuser',
        'https://www.amazon.com/s?k=reed+diffuser',
        'https://www.amazon.com/s?k=incense+sticks',
        'https://www.amazon.com/s?k=wax+melts',
        'https://www.amazon.com/s?k=aromatherapy+candles',
        'https://www.amazon.com/s?k=tea+light+candles',
        'https://www.amazon.com/s?k=pillar+candles',
        'https://www.amazon.com/s?k=candle+making+kit',
        'https://www.amazon.com/s?k=room+spray',
        'https://www.amazon.com/s?k=car+diffuser',
        'https://www.amazon.com/s?k=incense+holder',
        'https://www.amazon.com/s?k=candle+warmer',
        'https://www.amazon.com/s?k=soy+candles',
        'https://www.amazon.com/s?k=birthday+candles',
        'https://www.amazon.com/s?k=floating+candles',
        'https://www.amazon.com/s?k=citronella+candles',
        'https://www.amazon.com/s?k=candle+snuffer',
        'https://www.amazon.com/s?k=potpourri'
    ],

    # 43. SEWING & FABRIC - sewing machines, fabric, notions
    'sewing_fabric': [
        'https://www.amazon.com/s?k=sewing+machine',
        'https://www.amazon.com/s?k=fabric+by+yard',
        'https://www.amazon.com/s?k=sewing+thread',
        'https://www.amazon.com/s?k=sewing+needles',
        'https://www.amazon.com/s?k=fabric+scissors',
        'https://www.amazon.com/s?k=sewing+patterns',
        'https://www.amazon.com/s?k=pin+cushion',
        'https://www.amazon.com/s?k=seam+ripper',
        'https://www.amazon.com/s?k=measuring+tape',
        'https://www.amazon.com/s?k=thimble',
        'https://www.amazon.com/s?k=bobbin',
        'https://www.amazon.com/s?k=quilting+fabric',
        'https://www.amazon.com/s?k=embroidery+thread',
        'https://www.amazon.com/s?k=serger+machine',
        'https://www.amazon.com/s?k=cutting+mat',
        'https://www.amazon.com/s?k=rotary+cutter',
        'https://www.amazon.com/s?k=dress+form',
        'https://www.amazon.com/s?k=sewing+box',
        'https://www.amazon.com/s?k=fabric+glue',
        'https://www.amazon.com/s?k=iron+on+patches'
    ],

    # 44. PUZZLES & BRAIN TEASERS - puzzles for all ages
    'puzzles_brain_teasers': [
        'https://www.amazon.com/s?k=jigsaw+puzzle+1000',
        'https://www.amazon.com/s?k=3d+puzzle',
        'https://www.amazon.com/s?k=crossword+puzzle+book',
        'https://www.amazon.com/s?k=sudoku+book',
        'https://www.amazon.com/s?k=rubiks+cube',
        'https://www.amazon.com/s?k=brain+teaser+puzzles',
        'https://www.amazon.com/s?k=wooden+puzzle',
        'https://www.amazon.com/s?k=metal+puzzle',
        'https://www.amazon.com/s?k=puzzle+mat',
        'https://www.amazon.com/s?k=kids+puzzle',
        'https://www.amazon.com/s?k=puzzle+board',
        'https://www.amazon.com/s?k=puzzle+glue',
        'https://www.amazon.com/s?k=puzzle+frame',
        'https://www.amazon.com/s?k=logic+puzzles',
        'https://www.amazon.com/s?k=word+search+book',
        'https://www.amazon.com/s?k=tangram+puzzle',
        'https://www.amazon.com/s?k=puzzle+sorter',
        'https://www.amazon.com/s?k=escape+room+game',
        'https://www.amazon.com/s?k=puzzle+storage',
        'https://www.amazon.com/s?k=puzzle+roll'
    ],

    # 45. KITCHEN GADGETS - specialized kitchen tools
    'kitchen_gadgets': [
        'https://www.amazon.com/s?k=vegetable+spiralizer',
        'https://www.amazon.com/s?k=mandoline+slicer',
        'https://www.amazon.com/s?k=garlic+press',
        'https://www.amazon.com/s?k=can+opener',
        'https://www.amazon.com/s?k=pizza+cutter',
        'https://www.amazon.com/s?k=cheese+grater',
        'https://www.amazon.com/s?k=egg+slicer',
        'https://www.amazon.com/s?k=apple+corer',
        'https://www.amazon.com/s?k=avocado+slicer',
        'https://www.amazon.com/s?k=herb+scissors',
        'https://www.amazon.com/s?k=meat+thermometer',
        'https://www.amazon.com/s?k=kitchen+timer',
        'https://www.amazon.com/s?k=salad+spinner',
        'https://www.amazon.com/s?k=potato+masher',
        'https://www.amazon.com/s?k=citrus+juicer',
        'https://www.amazon.com/s?k=pasta+maker',
        'https://www.amazon.com/s?k=milk+frother',
        'https://www.amazon.com/s?k=ice+cream+scoop',
        'https://www.amazon.com/s?k=wine+opener',
        'https://www.amazon.com/s?k=kitchen+torch'
    ],

    # 46. EXERCISE EQUIPMENT - home gym equipment
    'exercise_equipment': [
        'https://www.amazon.com/s?k=treadmill',
        'https://www.amazon.com/s?k=elliptical+machine',
        'https://www.amazon.com/s?k=rowing+machine',
        'https://www.amazon.com/s?k=weight+bench',
        'https://www.amazon.com/s?k=pull+up+bar',
        'https://www.amazon.com/s?k=kettlebell',
        'https://www.amazon.com/s?k=medicine+ball',
        'https://www.amazon.com/s?k=foam+roller',
        'https://www.amazon.com/s?k=jump+rope',
        'https://www.amazon.com/s?k=ab+roller',
        'https://www.amazon.com/s?k=balance+ball',
        'https://www.amazon.com/s?k=barbell',
        'https://www.amazon.com/s?k=weight+plates',
        'https://www.amazon.com/s?k=squat+rack',
        'https://www.amazon.com/s?k=battle+ropes',
        'https://www.amazon.com/s?k=agility+ladder',
        'https://www.amazon.com/s?k=push+up+bars',
        'https://www.amazon.com/s?k=ankle+weights',
        'https://www.amazon.com/s?k=gym+mat',
        'https://www.amazon.com/s?k=suspension+trainer'
    ],

    # 47. RELIGIOUS & SPIRITUAL - religious items and books
    'religious_spiritual': [
        'https://www.amazon.com/s?k=bible',
        'https://www.amazon.com/s?k=cross+necklace',
        'https://www.amazon.com/s?k=prayer+beads',
        'https://www.amazon.com/s?k=religious+statue',
        'https://www.amazon.com/s?k=meditation+beads',
        'https://www.amazon.com/s?k=prayer+mat',
        'https://www.amazon.com/s?k=religious+candles',
        'https://www.amazon.com/s?k=rosary',
        'https://www.amazon.com/s?k=spiritual+books',
        'https://www.amazon.com/s?k=buddha+statue',
        'https://www.amazon.com/s?k=incense+burner',
        'https://www.amazon.com/s?k=singing+bowl',
        'https://www.amazon.com/s?k=chakra+stones',
        'https://www.amazon.com/s?k=tarot+cards',
        'https://www.amazon.com/s?k=crystal+healing',
        'https://www.amazon.com/s?k=prayer+journal',
        'https://www.amazon.com/s?k=religious+art',
        'https://www.amazon.com/s?k=sage+smudge',
        'https://www.amazon.com/s?k=holy+water',
        'https://www.amazon.com/s?k=devotional+books'
    ],

    # 48. ELECTRONIC ACCESSORIES - cables, adapters, accessories
    'electronic_accessories': [
        'https://www.amazon.com/s?k=hdmi+adapter',
        'https://www.amazon.com/s?k=usb+adapter',
        'https://www.amazon.com/s?k=ethernet+cable',
        'https://www.amazon.com/s?k=surge+protector',
        'https://www.amazon.com/s?k=extension+cord',
        'https://www.amazon.com/s?k=cable+management',
        'https://www.amazon.com/s?k=laptop+stand',
        'https://www.amazon.com/s?k=monitor+arm',
        'https://www.amazon.com/s?k=kvm+switch',
        'https://www.amazon.com/s?k=docking+station',
        'https://www.amazon.com/s?k=external+hard+drive',
        'https://www.amazon.com/s?k=usb+flash+drive',
        'https://www.amazon.com/s?k=card+reader',
        'https://www.amazon.com/s?k=cable+ties',
        'https://www.amazon.com/s?k=screen+cleaner',
        'https://www.amazon.com/s?k=keyboard+cover',
        'https://www.amazon.com/s?k=laptop+bag',
        'https://www.amazon.com/s?k=tablet+stand',
        'https://www.amazon.com/s?k=stylus+pen',
        'https://www.amazon.com/s?k=cable+organizer'
    ],

    # 49. MAPS & NAVIGATION - GPS, maps, compasses
    'maps_navigation': [
        'https://www.amazon.com/s?k=gps+navigation',
        'https://www.amazon.com/s?k=car+gps',
        'https://www.amazon.com/s?k=hiking+gps',
        'https://www.amazon.com/s?k=compass',
        'https://www.amazon.com/s?k=world+map',
        'https://www.amazon.com/s?k=atlas',
        'https://www.amazon.com/s?k=wall+map',
        'https://www.amazon.com/s?k=globe',
        'https://www.amazon.com/s?k=marine+gps',
        'https://www.amazon.com/s?k=topographic+maps',
        'https://www.amazon.com/s?k=road+atlas',
        'https://www.amazon.com/s?k=scratch+map',
        'https://www.amazon.com/s?k=map+pins',
        'https://www.amazon.com/s?k=navigation+charts',
        'https://www.amazon.com/s?k=map+case',
        'https://www.amazon.com/s?k=altimeter',
        'https://www.amazon.com/s?k=trail+maps',
        'https://www.amazon.com/s?k=vintage+maps',
        'https://www.amazon.com/s?k=map+compass',
        'https://www.amazon.com/s?k=gps+tracker'
    ],

    # 50. MAGAZINE SUBSCRIPTIONS - various magazine categories
    'magazines': [
        'https://www.amazon.com/s?k=magazine+subscription',
        'https://www.amazon.com/s?k=fashion+magazine',
        'https://www.amazon.com/s?k=sports+magazine',
        'https://www.amazon.com/s?k=science+magazine',
        'https://www.amazon.com/s?k=cooking+magazine',
        'https://www.amazon.com/s?k=travel+magazine',
        'https://www.amazon.com/s?k=business+magazine',
        'https://www.amazon.com/s?k=tech+magazine',
        'https://www.amazon.com/s?k=home+magazine',
        'https://www.amazon.com/s?k=fitness+magazine',
        'https://www.amazon.com/s?k=car+magazine',
        'https://www.amazon.com/s?k=photography+magazine',
        'https://www.amazon.com/s?k=news+magazine',
        'https://www.amazon.com/s?k=art+magazine',
        'https://www.amazon.com/s?k=music+magazine',
        'https://www.amazon.com/s?k=garden+magazine',
        'https://www.amazon.com/s?k=kids+magazine',
        'https://www.amazon.com/s?k=health+magazine',
        'https://www.amazon.com/s?k=lifestyle+magazine',
        'https://www.amazon.com/s?k=magazine+holder'
    ]
}

# Crawling Data

In [ ]:
def crawl(url_to_crawl):
  url = "https://scraper-api.decodo.com/v2/scrape"

  payload = {
        "url": url_to_crawl
  }

  headers = {
      "accept": "application/json",
      "content-type": "application/json",
      "authorization": decodo_auth
  }

  response = requests.post(url, json=payload, headers=headers)

  return response


In [ ]:
def get_product_image_links(images_list):
  image_links = []
  try:
    for img in images_list:
        if img.has_attr('src'):
            image_links.append(img['src'])

        elif img.has_attr(':src'):
            vue_src_code = img[':src']
            match = re.search(r"\|\|\s*'(.*?)'", vue_src_code)
            if match:
                image_links.append(match.group(1))

    unique_image_links = list(set(image_links))

    return unique_image_links
  except:
    return None

In [ ]:
def get_next_page(category_url):
  start_index = 1
  while True:
    crawled_page = crawl(f"{category_url}&page={start_index}")
    crawled_page_json = crawled_page.json()

    status_code = crawled_page_json['results'][0]['status_code']
    if status_code!=200:
      break

    html_string = crawled_page_json['results'][0]['content']
    soup = BeautifulSoup(html_string, 'html.parser')

    all_images = soup.find_all('img', class_='s-image')

    results = get_product_image_links(all_images)

    yield results

  start_index+=1

In [ ]:
data = []

In [ ]:
for category, category_urls in urls_to_crawl.items():
  print(f"Crawling {category}")
  image_links_crawled = 0

  for single_url in category_urls:
    for image_links in get_next_page(single_url):
      for image_link in image_links:
        data.append({
            'e-commerce_category': category,
            'image_link': image_link
        })

      image_links_crawled+= len(image_links)

      # Set a limit for results for testing
      if image_links_crawled >= 250:
        image_links_crawled = 0
        break
  print(f"Finished crawling links for {category}")
print("Finished crawling categories.")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("ecommerce_product_dataset.csv", index=False)

# Parameter and reading data

In [ ]:
dataset_csv_path = 'ecommerce_product_dataset.csv'
df = pd.read_csv(dataset_csv_path)
df = df.rename(columns={'e-commerce category': 'e-commerce_category'})
df.head()

In [ ]:
image_column_name = 'image_link'
label_column_name = 'e-commerce_category'

test_size = .2
num_labels = df[label_column_name].nunique()

model_name = 'google/vit-base-patch16-224'

# Clean Data

In [ ]:
# Remove urls for icons and ui elements.

urls_to_remove = ['https://m.media-amazon.com/images/I/11++B3A2NEL._SS200_.png',
                 'https://m.media-amazon.com/images/I/111mHoVK0kL._SS200_.png',
                 'https://m.media-amazon.com/images/G/01/nav2/dp/AddVehicle-v1._CB569730674_FMpng_SY80_.png',
                 'https://m.media-amazon.com/images/I/11lsIObTXaL._SS200_.png',
                  'https://m.media-amazon.com/images/I/11hfR5Cq9GL._SS200_.png',
                  'https://m.media-amazon.com/images/G/01/ap/comms-IOSXNT/common/logos/ap-logo-WPBKTI.svg',
                  'https://m.media-amazon.com/images/G/01/HST/ASINFaceout/Amazon_OM_RGB.svg']

df_filtered = df

for image_url in urls_to_remove:
  df_filtered = df_filtered[df_filtered['image_link'] != image_url]

In [ ]:
df = df_filtered

# Wrangle Data

## Label Encoder

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())
df['label'] = le.transform(df[label_column_name].tolist())

## Train/Test-Split

In [ ]:
df_train, df_test = train_test_split(df, test_size=test_size)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_train = df_train[['image_link', 'label']]
df_test = df_test[['image_link', 'label']]

## Convert to HuggingFace dataset

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# Image Processor

In [ ]:
image_processor = AutoImageProcessor.from_pretrained(model_name)

In [ ]:
def preprocess_function(examples):
    images = []
    for url in examples['image_link']:
        try:
            response = requests.get(url, stream=True, timeout=10)
            response.raise_for_status()

            image = Image.open(io.BytesIO(response.content)).convert("RGB")
            image = image.resize((224, 224))
            images.append(image)
        except Exception as e:
            print(f"Skipping URL due to error: {url} | Error: {e}")

    # Process only the successfully loaded images
    result = image_processor(images=images, return_tensors="pt")

    return result

In [ ]:
processed_train = train_dataset.map(preprocess_function,
                                    batched=True,
                                    batch_size=10,
                                    num_proc=1)
processed_test = test_dataset.map(preprocess_function,
                                    batched=True,
                                    batch_size=10,
                                    num_proc=1)

# Initialize Model

In [ ]:
model = AutoModelForImageClassification.from_pretrained(model_name,
                                                        num_labels=num_labels,
                                                        ignore_mismatched_sizes=True)


In [ ]:
number_of_layers = 0
for param in model.base_model.parameters():
  number_of_layers +=1
print(f"Number of layers: {number_of_layers}")

In [ ]:
layer_number = 0
for param in model.base_model.parameters():
  if layer_number >= number_of_layers-25:
    break
  number_of_layers+=1
  param.requires_grad=False

# Train Model

In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    report_to='none',
    fp16=True,
    learning_rate=2e-4,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=500,
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train,
    eval_dataset=processed_test,
    tokenizer=image_processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
model.config.id2label = {i: label for i, label in enumerate(le.classes_)}
model.config.label2id = {i: i for i, label in enumerate(le.classes_)}

In [ ]:
trainer.save_model('./ecommerce_classifier_model')
image_processor.save_pretrained('./ecommerce_classifier_model')

# Evaluate Model

In [ ]:
preds = trainer.predict(processed_train)
preds_max = np.argmax(preds[:3][0], axis=-1)
GT = df_train['label'].tolist()
print(classification_report(GT,preds_max))

In [ ]:
preds = trainer.predict(processed_test)
preds_max = np.argmax(preds[:3][0], axis=-1)
GT = df_test['label'].tolist()
print(classification_report(GT,preds_max))

# Model Inference

In [ ]:
clf = pipeline('image-classification',
               model="./ecommerce_classifier_model",
               image_processor="./ecommerce_classifier_model")

In [ ]:
test_img = Image.open('mechanical_pencil.jpg')

In [ ]:
result = clf(test_img)
print(result)

#### Push to Hugging Face

In [ ]:
trainer.push_to_hub("ecommerce_classification_model")
image_processor.push_to_hub("ecommerce_classification_model")